# 📊 Sales Data Analysis

## Comprehensive Sales Analysis and Visualization

**Author:** Data Science Portfolio  
**Date:** November 2025  
**Objective:** Analyze sales trends, identify top-performing products, and forecast revenue

---

## 1. Setup and Imports

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Statistics
from scipy import stats
from sklearn.linear_model import LinearRegression

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

print("✅ Libraries imported successfully")

## 2. Generate Sample Sales Data

For demonstration purposes, we'll generate a realistic sales dataset.

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

# Generate dates (2 years of daily data)
dates = pd.date_range(start='2023-01-01', end='2024-12-31', freq='D')
n_records = len(dates) * 5  # Multiple transactions per day

# Product categories and names
categories = ['Electronics', 'Clothing', 'Home & Garden', 'Sports', 'Books']
products = {
    'Electronics': ['Laptop', 'Smartphone', 'Tablet', 'Headphones', 'Smart Watch'],
    'Clothing': ['T-Shirt', 'Jeans', 'Jacket', 'Shoes', 'Dress'],
    'Home & Garden': ['Furniture', 'Decor', 'Kitchen Appliance', 'Bedding', 'Garden Tools'],
    'Sports': ['Running Shoes', 'Yoga Mat', 'Dumbbell', 'Bicycle', 'Tennis Racket'],
    'Books': ['Fiction', 'Non-Fiction', 'Educational', 'Comics', 'Magazines']
}

# Regions
regions = ['North', 'South', 'East', 'West', 'Central']

# Generate data
data = {
    'Date': np.random.choice(dates, n_records),
    'Category': np.random.choice(categories, n_records),
    'Region': np.random.choice(regions, n_records),
    'Units_Sold': np.random.randint(1, 20, n_records),
    'Unit_Price': np.random.uniform(10, 1000, n_records),
    'Customer_Age': np.random.randint(18, 70, n_records),
    'Customer_Segment': np.random.choice(['Individual', 'Corporate', 'Government'], n_records, p=[0.7, 0.25, 0.05])
}

# Create DataFrame
df = pd.DataFrame(data)

# Add product names based on category
df['Product'] = df['Category'].apply(lambda x: np.random.choice(products[x]))

# Calculate revenue
df['Revenue'] = df['Units_Sold'] * df['Unit_Price']

# Add some seasonality (Q4 boost)
df['Month'] = pd.to_datetime(df['Date']).dt.month
df.loc[df['Month'].isin([11, 12]), 'Revenue'] *= 1.3

# Sort by date
df = df.sort_values('Date').reset_index(drop=True)

print(f"✅ Generated {len(df):,} sales records")
print(f"📅 Date range: {df['Date'].min()} to {df['Date'].max()}")
print(f"💰 Total revenue: ${df['Revenue'].sum():,.2f}")

## 3. Data Overview

In [ ]:
# Display first few rows
print("📋 First 10 rows:")
df.head(10)

In [ ]:
# Data info
print("ℹ️ Dataset Information:")
df.info()

In [ ]:
# Statistical summary
print("📊 Statistical Summary:")
df.describe()

## 4. Data Cleaning and Preparation

In [ ]:
# Check for missing values
print("🔍 Missing values:")
print(df.isnull().sum())

# Check for duplicates
duplicates = df.duplicated().sum()
print(f"\n🔄 Duplicate rows: {duplicates}")

# Convert Date to datetime if not already
df['Date'] = pd.to_datetime(df['Date'])

# Extract time features
df['Year'] = df['Date'].dt.year
df['Quarter'] = df['Date'].dt.quarter
df['Month_Name'] = df['Date'].dt.strftime('%B')
df['Day_of_Week'] = df['Date'].dt.day_name()

print("\n✅ Data cleaning completed")

## 5. Exploratory Data Analysis (EDA)

### 5.1 Revenue Trends Over Time

In [ ]:
# Monthly revenue
monthly_revenue = df.groupby(df['Date'].dt.to_period('M'))['Revenue'].sum().reset_index()
monthly_revenue['Date'] = monthly_revenue['Date'].dt.to_timestamp()

# Create interactive plot
fig = px.line(monthly_revenue, x='Date', y='Revenue',
              title='Monthly Revenue Trend',
              labels={'Revenue': 'Revenue ($)'},
              template='plotly_white')

fig.update_traces(line_color='#2E86AB', line_width=3)
fig.update_layout(hovermode='x unified', height=500)
fig.show()

print(f"📈 Average monthly revenue: ${monthly_revenue['Revenue'].mean():,.2f}")
print(f"📊 Revenue growth: {((monthly_revenue['Revenue'].iloc[-1] / monthly_revenue['Revenue'].iloc[0]) - 1) * 100:.1f}%")

### 5.2 Revenue by Category

In [ ]:
# Revenue by category
category_revenue = df.groupby('Category')['Revenue'].sum().sort_values(ascending=False)

# Create pie chart
fig = px.pie(values=category_revenue.values, names=category_revenue.index,
             title='Revenue Distribution by Category',
             hole=0.4,
             color_discrete_sequence=px.colors.qualitative.Set3)

fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

print("💰 Revenue by Category:")
for cat, rev in category_revenue.items():
    print(f"  {cat}: ${rev:,.2f} ({rev/category_revenue.sum()*100:.1f}%)")

### 5.3 Top Products

In [ ]:
# Top 10 products by revenue
top_products = df.groupby('Product')['Revenue'].sum().sort_values(ascending=False).head(10)

# Create bar chart
fig = px.bar(x=top_products.values, y=top_products.index,
             orientation='h',
             title='Top 10 Products by Revenue',
             labels={'x': 'Revenue ($)', 'y': 'Product'},
             color=top_products.values,
             color_continuous_scale='Viridis')

fig.update_layout(showlegend=False, height=500)
fig.show()

print("🏆 Top 5 Products:")
for i, (prod, rev) in enumerate(top_products.head().items(), 1):
    print(f"  {i}. {prod}: ${rev:,.2f}")

### 5.4 Regional Performance

In [ ]:
# Revenue and units by region
region_stats = df.groupby('Region').agg({
    'Revenue': 'sum',
    'Units_Sold': 'sum'
}).sort_values('Revenue', ascending=False)

# Create subplot
fig = make_subplots(rows=1, cols=2, 
                    subplot_titles=('Revenue by Region', 'Units Sold by Region'))

fig.add_trace(
    go.Bar(x=region_stats.index, y=region_stats['Revenue'], 
           name='Revenue', marker_color='lightblue'),
    row=1, col=1
)

fig.add_trace(
    go.Bar(x=region_stats.index, y=region_stats['Units_Sold'], 
           name='Units', marker_color='lightcoral'),
    row=1, col=2
)

fig.update_layout(showlegend=False, height=400, title_text='Regional Performance')
fig.show()

print("🗺️ Regional Performance:")
print(region_stats)

### 5.5 Quarterly Analysis

In [ ]:
# Quarterly revenue by year
quarterly = df.groupby(['Year', 'Quarter'])['Revenue'].sum().reset_index()
quarterly['Period'] = quarterly['Year'].astype(str) + '-Q' + quarterly['Quarter'].astype(str)

# Create grouped bar chart
fig = px.bar(quarterly, x='Period', y='Revenue', color='Year',
             title='Quarterly Revenue Comparison',
             labels={'Revenue': 'Revenue ($)'},
             barmode='group',
             color_discrete_sequence=['#A8DADC', '#457B9D'])

fig.update_layout(height=450)
fig.show()

# Calculate Q4 boost
q4_avg = quarterly[quarterly['Quarter'] == 4]['Revenue'].mean()
other_avg = quarterly[quarterly['Quarter'] != 4]['Revenue'].mean()
q4_boost = ((q4_avg / other_avg) - 1) * 100

print(f"📊 Q4 Performance Boost: {q4_boost:.1f}% above average")

### 5.6 Customer Segment Analysis

In [ ]:
# Segment analysis
segment_stats = df.groupby('Customer_Segment').agg({
    'Revenue': ['sum', 'mean', 'count']
}).round(2)

segment_stats.columns = ['Total Revenue', 'Avg Transaction', 'Transaction Count']
segment_stats = segment_stats.sort_values('Total Revenue', ascending=False)

print("👥 Customer Segment Analysis:")
print(segment_stats)

# Visualize
fig = px.sunburst(df, path=['Customer_Segment', 'Category'], 
                  values='Revenue',
                  title='Revenue Breakdown: Segment → Category',
                  color='Revenue',
                  color_continuous_scale='RdYlGn')

fig.update_layout(height=600)
fig.show()

### 5.7 Day of Week Analysis

In [ ]:
# Revenue by day of week
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dow_revenue = df.groupby('Day_of_Week')['Revenue'].sum().reindex(day_order)

# Create bar chart
fig = px.bar(x=dow_revenue.index, y=dow_revenue.values,
             title='Revenue by Day of Week',
             labels={'x': 'Day', 'y': 'Revenue ($)'},
             color=dow_revenue.values,
             color_continuous_scale='Blues')

fig.update_layout(showlegend=False, height=450)
fig.show()

best_day = dow_revenue.idxmax()
worst_day = dow_revenue.idxmin()
print(f"🔝 Best day: {best_day} (${dow_revenue[best_day]:,.2f})")
print(f"📉 Slowest day: {worst_day} (${dow_revenue[worst_day]:,.2f})")

## 6. Correlation Analysis

In [ ]:
# Select numeric columns
numeric_cols = ['Units_Sold', 'Unit_Price', 'Revenue', 'Customer_Age']
correlation = df[numeric_cols].corr()

# Create heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Heatmap', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("🔗 Key Correlations:")
print(correlation['Revenue'].sort_values(ascending=False))

## 7. Revenue Forecasting

In [ ]:
# Prepare data for forecasting
monthly_data = df.groupby(df['Date'].dt.to_period('M'))['Revenue'].sum().reset_index()
monthly_data['Date'] = monthly_data['Date'].dt.to_timestamp()
monthly_data['Month_Num'] = range(len(monthly_data))

# Train linear regression model
X = monthly_data[['Month_Num']].values
y = monthly_data['Revenue'].values

model = LinearRegression()
model.fit(X, y)

# Make predictions
monthly_data['Predicted'] = model.predict(X)

# Forecast next 6 months
future_months = np.arange(len(monthly_data), len(monthly_data) + 6).reshape(-1, 1)
future_predictions = model.predict(future_months)

# Calculate R-squared
r2 = model.score(X, y)

# Visualize
fig = go.Figure()

fig.add_trace(go.Scatter(x=monthly_data['Date'], y=monthly_data['Revenue'],
                         mode='lines+markers', name='Actual',
                         line=dict(color='#2E86AB', width=2)))

fig.add_trace(go.Scatter(x=monthly_data['Date'], y=monthly_data['Predicted'],
                         mode='lines', name='Trend',
                         line=dict(color='#A23B72', width=2, dash='dash')))

# Add future predictions
last_date = monthly_data['Date'].iloc[-1]
future_dates = pd.date_range(start=last_date + pd.DateOffset(months=1), periods=6, freq='MS')

fig.add_trace(go.Scatter(x=future_dates, y=future_predictions,
                         mode='lines+markers', name='Forecast',
                         line=dict(color='#F18F01', width=2, dash='dot')))

fig.update_layout(title='Revenue Forecast (Linear Regression)',
                  xaxis_title='Date',
                  yaxis_title='Revenue ($)',
                  hovermode='x unified',
                  height=500)

fig.show()

print(f"📊 Model R-squared: {r2:.4f}")
print(f"📈 Trend: ${model.coef_[0]:,.2f} per month")
print(f"\n🔮 6-Month Forecast:")
for date, pred in zip(future_dates, future_predictions):
    print(f"  {date.strftime('%B %Y')}: ${pred:,.2f}")

## 8. Key Insights and Recommendations

In [ ]:
# Calculate key metrics
total_revenue = df['Revenue'].sum()
total_units = df['Units_Sold'].sum()
avg_transaction = df['Revenue'].mean()
top_category = df.groupby('Category')['Revenue'].sum().idxmax()
top_region = df.groupby('Region')['Revenue'].sum().idxmax()

# YoY growth
revenue_2023 = df[df['Year'] == 2023]['Revenue'].sum()
revenue_2024 = df[df['Year'] == 2024]['Revenue'].sum()
yoy_growth = ((revenue_2024 / revenue_2023) - 1) * 100

print("="*60)
print("📊 KEY PERFORMANCE INDICATORS")
print("="*60)
print(f"\n💰 Total Revenue: ${total_revenue:,.2f}")
print(f"📦 Total Units Sold: {total_units:,}")
print(f"💵 Average Transaction Value: ${avg_transaction:,.2f}")
print(f"📈 YoY Revenue Growth: {yoy_growth:.1f}%")
print(f"\n🏆 Top Performing Category: {top_category}")
print(f"🗺️ Top Performing Region: {top_region}")
print(f"👥 Largest Customer Segment: {df['Customer_Segment'].value_counts().idxmax()}")

print("\n" + "="*60)
print("💡 KEY INSIGHTS")
print("="*60)
print(f"\n1. Strong Q4 Performance: {q4_boost:.1f}% above average")
print(f"2. {top_category} category dominates with {category_revenue[top_category]/total_revenue*100:.1f}% of revenue")
print(f"3. {top_region} region leads in sales performance")
print(f"4. Positive growth trend: ${model.coef_[0]:,.0f}/month")
print(f"5. Individual customers represent 70% of transactions")

print("\n" + "="*60)
print("🎯 RECOMMENDATIONS")
print("="*60)
print("\n1. Focus marketing efforts on Q4 to maximize seasonal boost")
print(f"2. Expand {top_category} inventory based on strong demand")
print(f"3. Replicate {top_region} region's strategies in other areas")
print("4. Develop loyalty programs for individual customers")
print("5. Optimize pricing strategies for high-margin products")
print("6. Increase promotional activities on slower weekdays")

## 9. Save Results

In [ ]:
# Save processed data
# df.to_csv('sales_data_processed.csv', index=False)
# print("✅ Processed data saved to 'sales_data_processed.csv'")

# Save summary statistics
summary = {
    'Total Revenue': total_revenue,
    'Total Units': total_units,
    'Avg Transaction': avg_transaction,
    'YoY Growth': yoy_growth,
    'Top Category': top_category,
    'Top Region': top_region
}

# summary_df = pd.DataFrame([summary])
# summary_df.to_csv('sales_summary.csv', index=False)
# print("✅ Summary saved to 'sales_summary.csv'")

print("\n✅ Analysis completed successfully!")

---

## Conclusion

This analysis provides comprehensive insights into sales performance, identifying key trends, top-performing products and regions, and revenue forecasts. The findings can be used to optimize inventory, marketing strategies, and resource allocation for maximum profitability.

**Next Steps:**
1. Implement recommendations
2. Monitor KPIs monthly
3. Conduct customer satisfaction surveys
4. Explore advanced forecasting models (ARIMA, Prophet)
5. Develop automated dashboards for real-time monitoring